In [17]:
import boto3
import json
data={
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Principal": {
        "Service": "comprehend.amazonaws.com"
      },
      "Action": "sts:AssumeRole"
    }
  ]
}

client=boto3.client('iam')

response = client.create_role(
    RoleName='ComprheRole',
    AssumeRolePolicyDocument=json.dumps(data)
)

In [18]:

response = client.attach_role_policy(
    RoleName='ComprheRole',
    PolicyArn='arn:aws:iam::aws:policy/service-role/ComprehendDataAccessRolePolicy'
)

In [86]:
import boto3

client=boto3.client('s3')
response = client.create_bucket(
    Bucket='comprehendjob-query',
    
)
print(response)

{'ResponseMetadata': {'RequestId': '39478EA490CBD7B6', 'HostId': 'XxvW+iuiOgHEylMVUM3B6bEk+eC0e4SiJm4rC7q8Aj+P+K1N92bmD60MCkRjU12c7BR899gRfAs=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'XxvW+iuiOgHEylMVUM3B6bEk+eC0e4SiJm4rC7q8Aj+P+K1N92bmD60MCkRjU12c7BR899gRfAs=', 'x-amz-request-id': '39478EA490CBD7B6', 'date': 'Fri, 16 Oct 2020 04:45:26 GMT', 'location': '/comprehendjob-query', 'content-length': '0', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'Location': '/comprehendjob-query'}


In [90]:
import boto3

client=boto3.client('comprehend')
response = client.start_sentiment_detection_job(
    InputDataConfig={
        'S3Uri': 's3://senti-analysis-in/amazon-reviews.csv',
        'InputFormat':'ONE_DOC_PER_LINE'
    },
    OutputDataConfig={
        'S3Uri': 's3://senti-analysis-out'
    },
    DataAccessRoleArn='arn:aws:iam::141292516170:role/ComprheRole',
    JobName='comp-sentiment-analysis',
    LanguageCode='en'
)
print(response)

{'JobId': '12f3967489b48b38c3a1eea8979455f7', 'JobStatus': 'SUBMITTED', 'ResponseMetadata': {'RequestId': '4229fb6b-220e-49f3-87a1-f95c4f945f16', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '4229fb6b-220e-49f3-87a1-f95c4f945f16', 'content-type': 'application/x-amz-json-1.1', 'content-length': '68', 'date': 'Fri, 16 Oct 2020 06:45:46 GMT'}, 'RetryAttempts': 0}}


In [81]:
import boto3
s3 = boto3.resource('s3')
s3.meta.client.upload_file('C:/Users/MY PC/Documents/docs/output', 'comprehendjob-senti-res', 'output')


In [67]:
import boto3
import json
data={
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Principal": {
        "Service": "glue.amazonaws.com"
      },
      "Action": "sts:AssumeRole"
    }
  ]
}

client=boto3.client('iam')

response = client.create_role(
    RoleName='GlueRole1',
    AssumeRolePolicyDocument=json.dumps(data)
)

In [68]:
import json
data={
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "s3:GetObject",
                "s3:PutObject"
            ],
            "Resource": [
                "arn:aws:s3:::comprehendjob-bucketres/output"
            ]
        }
    ]
}
response = client.create_policy(
    PolicyName='glue_s3policy',
    PolicyDocument=json.dumps(data)

)

In [69]:
response = client.attach_role_policy(
    RoleName='GlueRole1',
    PolicyArn='arn:aws:iam::141292516170:policy/glue_s3policy'
)

In [84]:
import boto3

client = boto3.client('glue')
response = client.create_crawler(
    Name='comprehend-analysis-crawler2',
    Role='GlueRole',
    DatabaseName='comprehend-results1',
    Targets={
        'S3Targets': [
            {
                'Path': 's3://comprehendjob-senti-res/',
                
            },
        ]
    },
    
)
print(response)

{'ResponseMetadata': {'RequestId': '381f6b0e-63d6-4664-9a5b-9030ea2d9c75', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Thu, 15 Oct 2020 10:13:38 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '2', 'connection': 'keep-alive', 'x-amzn-requestid': '381f6b0e-63d6-4664-9a5b-9030ea2d9c75'}, 'RetryAttempts': 0}}


In [83]:
import boto3
client= boto3.client('glue')
response = client.create_database(
    DatabaseInput={
        'Name': 'comprehend-results1',
       
        }
    
)

In [85]:
import boto3

client=boto3.client('glue')
response = client.start_crawler(
    Name='comprehend-analysis-crawler2'
)

In [48]:
import boto3

client=boto3.client('athena')

response = client.create_named_query(
    Name='sql',
    Database='comprehend-results',
    QueryString='''CREATE TABLE sentiment_results_final AS
SELECT file, line, sentiment,
sentimentscore.mixed AS mixed,
sentimentscore.negative AS negative,
sentimentscore.neutral AS neutral,
sentimentscore.positive AS positive
FROM sentiment_results'''
)
print(response)

{'NamedQueryId': '70fca7b1-9990-4061-bc30-be8d38325858', 'ResponseMetadata': {'RequestId': 'ad9c0d53-3edf-4d0e-becb-bead08b51e6e', 'HTTPStatusCode': 200, 'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1', 'date': 'Tue, 13 Oct 2020 10:27:12 GMT', 'x-amzn-requestid': 'ad9c0d53-3edf-4d0e-becb-bead08b51e6e', 'content-length': '55', 'connection': 'keep-alive'}, 'RetryAttempts': 0}}


In [50]:
response = client.get_named_query(
    NamedQueryId='70fca7b1-9990-4061-bc30-be8d38325858'
)
print(response)


{'NamedQuery': {'Name': 'sql', 'Database': 'comprehend-results', 'QueryString': 'CREATE TABLE sentiment_results_final AS\nSELECT file, line, sentiment,\nsentimentscore.mixed AS mixed,\nsentimentscore.negative AS negative,\nsentimentscore.neutral AS neutral,\nsentimentscore.positive AS positive\nFROM sentiment_results', 'NamedQueryId': '70fca7b1-9990-4061-bc30-be8d38325858', 'WorkGroup': 'primary'}, 'ResponseMetadata': {'RequestId': 'b4e45a11-f4b0-4f24-9893-80b3d55d4475', 'HTTPStatusCode': 200, 'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1', 'date': 'Tue, 13 Oct 2020 10:37:35 GMT', 'x-amzn-requestid': 'b4e45a11-f4b0-4f24-9893-80b3d55d4475', 'content-length': '391', 'connection': 'keep-alive'}, 'RetryAttempts': 0}}


In [88]:
import boto3

client=boto3.client('athena')

response = client.start_query_execution(
    QueryString='''CREATE TABLE sentiment_results_final AS
SELECT file, line, sentiment,
sentimentscore.mixed AS mixed,
sentimentscore.negative AS negative,
sentimentscore.neutral AS neutral,
sentimentscore.positive AS positive
FROM comprehendjob_senti_res''',
    QueryExecutionContext={
        'Database': 'comprehend-results1'
    },
    ResultConfiguration={
        'OutputLocation': 's3://comprehendjob-query/',
        'EncryptionConfiguration': {
            'EncryptionOption': 'SSE_S3'
        }
    }
)
print(response)

{'QueryExecutionId': '902581e3-fef2-4dfe-bc16-9cca74d264b2', 'ResponseMetadata': {'RequestId': 'dc718524-6df4-4e5a-8283-138fb24cecd0', 'HTTPStatusCode': 200, 'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1', 'date': 'Fri, 16 Oct 2020 04:47:29 GMT', 'x-amzn-requestid': 'dc718524-6df4-4e5a-8283-138fb24cecd0', 'content-length': '59', 'connection': 'keep-alive'}, 'RetryAttempts': 0}}


In [89]:
response = client.get_query_results(
    QueryExecutionId='902581e3-fef2-4dfe-bc16-9cca74d264b2',
   
)
print(response)

{'UpdateCount': 36, 'ResultSet': {'Rows': [], 'ResultSetMetadata': {'ColumnInfo': [{'CatalogName': 'hive', 'SchemaName': '', 'TableName': '', 'Name': 'rows', 'Label': 'rows', 'Type': 'bigint', 'Precision': 19, 'Scale': 0, 'Nullable': 'UNKNOWN', 'CaseSensitive': False}]}}, 'ResponseMetadata': {'RequestId': 'e6086e94-eacc-4e7c-b796-2ab6af988c45', 'HTTPStatusCode': 200, 'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1', 'date': 'Fri, 16 Oct 2020 04:50:02 GMT', 'x-amzn-requestid': 'e6086e94-eacc-4e7c-b796-2ab6af988c45', 'content-length': '445', 'connection': 'keep-alive'}, 'RetryAttempts': 0}}


In [79]:
import boto3
import json

data={
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": "s3-outposts:*",
            "Resource": "*"
        },
        {
            "Effect": "Allow",
            "Action": [
                "datasync:ListTasks",
                "datasync:ListLocations",
                "datasync:DescribeTask",
                "datasync:DescribeLocation*"
            ],
            "Resource": "*"
        },
        {
            "Effect": "Allow",
            "Action": [
                "ec2:DescribeVpcs",
                "ec2:DescribeSubnets",
                "ec2:DescribeSecurityGroups",
                "ec2:DescribeNetworkInterfaces"
            ],
            "Resource": "*"
        },
        {
            "Effect": "Allow",
            "Action": [
                "outposts:ListOutposts",
                "outposts:GetOutpost"
            ],
            "Resource": "*"
        }
    ]
}
client=boto3.client('s3')
response = client.put_bucket_policy(
    Bucket='comprehendjob-bucketres',
    Policy=json.dumps(data)

SyntaxError: unexpected EOF while parsing (<ipython-input-79-01095f8762b0>, line 45)